In [4]:
import pickle
from pathlib import Path
import os
import json

In [5]:
with open('data.json', 'r') as f:
    all_data = json.load(f)
#all_data = {}
for p in Path('data').glob('*.p'):
    with open(p, 'rb') as f:
        obj = pickle.load(f)
    site_data = {}
    for k, v in obj.items():
        if isinstance(k, str):
            site_data[k] = v
        else:
            year, month = k
            if year not in site_data:
                site_data[year] = {}
            year_data = site_data[year]
            
#             year_data = site_data.setdefault(year, {})

#             year_data = site_data.get(year, {})
#             site_data[year] = year_data
            
            year_data[str(month)] = dict(sorted(v.items()))
    
    site_data = dict(sorted(site_data.items()))
    file_name = os.path.splitext(os.path.basename(p))[0]
    state, city, station = file_name.split('_')
    
    if state not in all_data:
        all_data[state] = {}
    state_data = all_data[state]
    
    if city not in state_data:
        state_data[city] = {}
    city_data = state_data[city]
    
    city_data[station] = site_data

In [8]:
with open('data_all.json', 'w') as f:
    json.dump(all_data, f)

In [10]:
for state in all_data:
    state_data = all_data[state]
    for city in state_data:
        city_data = state_data[city]
        for station in city_data:
            for year, year_data in city_data[station].items():
                if year == 'done':
                    continue
                for month, month_data in year_data.items():
                    if month == 'done':
                        continue
                    for day, day_data in month_data.items():
                        if day == 'done' or len(day_data) == 0:
                            continue
                        try:
                            float(day_data['min_PM25'])
                        except ValueError:
                            print('|'.join([state, city, station, year, month, day]))

Maharashtra|Mumbai|Bandra, Mumbai - MPCB|2019|10|26
Maharashtra|Mumbai|Bandra, Mumbai - MPCB|2019|10|27
Maharashtra|Mumbai|Bandra, Mumbai - MPCB|2019|11|14
Maharashtra|Mumbai|Bandra, Mumbai - MPCB|2019|11|15
Maharashtra|Nashik|Gangapur Road, Nashik - MPCB|2018|11|6
Maharashtra|Chandrapur|Chandrapur, Chandrapur - MPCB|2019|5|26
Maharashtra|Chandrapur|Chandrapur, Chandrapur - MPCB|2019|5|27
Maharashtra|Chandrapur|MIDC Khutala, Chandrapur - MPCB|2018|6|28
Delhi|Delhi|IHBAS, Dilshad Garden, Delhi - CPCB|2018|12|29
Delhi|Delhi|IHBAS, Dilshad Garden, Delhi - CPCB|2018|12|30
Delhi|Delhi|Punjabi Bagh, Delhi - DPCC|2018|1|18
Delhi|Delhi|Punjabi Bagh, Delhi - DPCC|2018|1|19
Delhi|Delhi|R K Puram, Delhi - DPCC|2019|11|18
Delhi|Delhi|R K Puram, Delhi - DPCC|2019|11|19
Delhi|Delhi|Nehru Nagar, Delhi - DPCC|2018|11|13
Delhi|Delhi|Nehru Nagar, Delhi - DPCC|2018|11|14
Delhi|Delhi|Nehru Nagar, Delhi - DPCC|2018|11|15
Delhi|Delhi|Ashok Vihar, Delhi - DPCC|2018|2|12
Delhi|Delhi|Ashok Vihar, Delhi - DPCC|

In [100]:
failed_data[0]['min_PM25']

'1'

In [91]:
with open('data_all.json', 'r') as f:
    all_data = json.load(f)
    
with open("failed.txt", "r") as f:
    lines = f.readlines()
    
with open('failed_data.json','r') as f:
    failed_data = json.load(f)
    
for line, failed_day_data in zip(lines, failed_data):
    state, city, station, year, month, day = line[:-1].split('|')
    all_data[state][city][station][year][month][day] = failed_day_data
    

In [105]:
# for loop over all_data
# append each entry to a list of rows
all_data_list = []
for state in all_data:
    state_data = all_data[state]
    for city in state_data:
        city_data = state_data[city]
        for station in city_data:
            for year, year_data in city_data[station].items():
                if year == 'done':
                    continue
                for month, month_data in year_data.items():
                    if month == 'done':
                        continue
                    for day, day_data in month_data.items():
                        if day == 'done' or len(day_data) == 0:
                            continue
                        all_data_list.append([state, city, station, year, month, day, day_data['min_PM25'], day_data['max_PM25'], day_data['avg_PM25']])
                        

In [109]:
import pandas as pd
df = pd.DataFrame(all_data_list, columns = ['state','city','station','year','month','day','min PM25','max PM25', 'avg PM25'])
df.to_csv('all_data.csv')